# Exploratory Data Analysis — Video Game Sales with Ratings

**Dataset**: [rush4ratio / Video Game Sales with Ratings]  
**Objetivo**: preparar datos limpios, visualizar patrones clave y extraer conclusiones ejecutivas para el CEO.

> **Cómo usar**: Coloca el CSV del dataset en `/mnt/data/` y ejecuta las celdas en orden.  
> Este notebook **no usa seaborn** (requisito de presentación) y se apoya en **pandas** y **matplotlib**.

In [2]:
# Librerías a usar
import os
import sys
import math
import json
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)
print("Versions -> pandas:", pd.__version__, "| numpy:", np.__version__)

Versions -> pandas: 2.3.2 | numpy: 2.3.2


## 1) Carga de datos

In [6]:
from pathlib import Path

# Soporta varios nombres comunes del archivo en este dataset
candidate_files = [
    "Video_Games_Sales_as_at_22_Dec_2016.csv",
    "Video_Games_Sales_as_at_22_Dec_2016.csv.zip",
    "vgsales.csv",
    "vgsales-with-ratings.csv"
]

data_path = None
for name in candidate_files:
    p = Path("data/") / name
    if p.exists():
        data_path = p
        break

if data_path is None:
    print("❗ Noo se encontró el CSV. Copia el archivo en /mnt/data y re-ejecuta esta celda.")
else:
    print("✅ Archivoo detectado:", data_path)

✅ Archivoo detectado: data\Video_Games_Sales_as_at_22_Dec_2016.csv


## 2) Exploración inicial

In [7]:
if 'df' in globals():
    # Nombres y tipos
    display(pd.DataFrame({
        "column": df.columns,
        "dtype": [df[c].dtype for c in df.columns],
        "n_null": [df[c].isna().sum() for c in df.columns],
        "n_unique": [df[c].nunique(dropna=True) for c in df.columns]
    }))

    # Vista rápida de estadísticas numéricas
    display(df.describe(include=[np.number]).T)

    # Vista de categóricas
    cat_cols = [c for c in df.columns if df[c].dtype == "object"]
    display(df[cat_cols].describe().T)

## 3) Limpieza y preprocesado

In [8]:
if 'df' in globals():
    df_clean = df.copy()

    # Estandarizar nombres de columnas (lowercase, snake_case)
    df_clean.columns = (
        df_clean.columns
        .str.strip()
        .str.replace("\s+", "_", regex=True)
        .str.replace("[^0-9a-zA-Z_]", "", regex=True)
        .str.lower()
    )

    # Renombrados útiles si existen
    rename_map = {
        "name": "title",
        "platform": "platform",
        "year_of_release": "year",
        "genre": "genre",
        "publisher": "publisher",
        "na_sales": "na_sales",
        "eu_sales": "eu_sales",
        "jp_sales": "jp_sales",
        "other_sales": "other_sales",
        "global_sales": "global_sales",
        "critic_score": "critic_score",
        "critic_count": "critic_count",
        "user_score": "user_score",
        "user_count": "user_count",
        "developer": "developer",
        "rating": "esrb_rating"
    }
    df_clean = df_clean.rename(columns={k:v for k,v in rename_map.items() if k in df_clean.columns})

    # Limpiar año
    if "year" in df_clean.columns:
        # Algunos años vienen como float o con NaN
        def coerce_year(x):
            try:
                y = int(float(x))
                return np.nan if y < 1970 or y > 2030 else y
            except:
                return np.nan
        df_clean["year"] = df_clean["year"].apply(coerce_year)

    # Normalizar plataforma (ej: 'PS4', 'PlayStation 4' -> 'PS4')
    if "platform" in df_clean.columns:
        platform_map = {
            "PlayStation 4": "PS4",
            "PlayStation 3": "PS3",
            "PlayStation 2": "PS2",
            "PlayStation": "PS1",
            "Xbox 360": "X360",
            "Xbox One": "XOne",
            "Nintendo DS": "DS",
            "Nintendo 3DS": "3DS",
            "Nintendo Switch": "NS"
        }
        df_clean["platform"] = df_clean["platform"].replace(platform_map)
        df_clean["platform"] = df_clean["platform"].astype(str).str.upper().str.strip()

    # User score a numérico si está en escala 0-10 (a veces viene como string 'tbd')
    if "user_score" in df_clean.columns:
        def parse_user_score(x):
            try:
                v = float(x)
                return v
            except:
                return np.nan
        df_clean["user_score"] = df_clean["user_score"].apply(parse_user_score)

    # Cálculo de ventas limpias
    for c in ["na_sales", "eu_sales", "jp_sales", "other_sales", "global_sales"]:
        if c in df_clean.columns:
            df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")

    # Eliminar duplicados exactos
    before = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    removed_dups = before - len(df_clean)

    print(f"Limpieza realizada. Duplicados eliminados: {removed_dups}. Filas actuales: {len(df_clean)}")
    display(df_clean.head())

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\yeder\AppData\Local\Temp\ipykernel_21160\4137126519.py:8: SyntaxWarning: invalid escape sequence '\s'
  .str.replace("\s+", "_", regex=True)


## 4) Análisis exploratorio (EDA)

In [ ]:
if 'df_clean' in globals():
    # Distribución por año
    if "year" in df_clean.columns:
        year_counts = df_clean['year'].dropna().astype(int).value_counts().sort_index()
        display(year_counts.head())

        plt.figure(figsize=(10,5))
        year_counts.plot(kind='bar')
        plt.title("Número de lanzamientos por año")
        plt.xlabel("Año")
        plt.ylabel("Número de juegos")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [ ]:
if 'df_clean' in globals() and set(["platform","global_sales"]).issubset(df_clean.columns):
    top_platforms = (
        df_clean.groupby("platform")["global_sales"]
        .sum()
        .sort_values(ascending=False)
        .head(15)
    )
    display(top_platforms)

    plt.figure(figsize=(10,5))
    top_platforms.plot(kind='bar')
    plt.title("Top plataformas por ventas globales (M de copias)")
    plt.xlabel("Plataforma")
    plt.ylabel("Ventas globales (millones)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
if 'df_clean' in globals() and set(["genre","global_sales"]).issubset(df_clean.columns):
    genre_sales = (
        df_clean.groupby("genre")["global_sales"]
        .sum()
        .sort_values(ascending=False)
    )
    display(genre_sales)

    plt.figure(figsize=(8,5))
    genre_sales.plot(kind='bar')
    plt.title("Ventas globales por género (M de copias)")
    plt.xlabel("Género")
    plt.ylabel("Ventas globales (millones)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
if 'df_clean' in globals() and set(["publisher","global_sales"]).issubset(df_clean.columns):
    top_publishers = (
        df_clean.groupby("publisher")["global_sales"]
        .sum()
        .sort_values(ascending=False)
        .head(15)
    )
    display(top_publishers)

    plt.figure(figsize=(10,5))
    top_publishers.plot(kind='bar')
    plt.title("Top 15 publishers por ventas globales (M de copias)")
    plt.xlabel("Publisher")
    plt.ylabel("Ventas globales (millones)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
if 'df_clean' in globals() and set(["critic_score","user_score","global_sales"]).issubset(df_clean.columns):
    # Correlaciones numéricas
    num_cols = df_clean.select_dtypes(include=[np.number]).columns
    corr = df_clean[num_cols].corr()
    display(corr)

    plt.figure(figsize=(8,6))
    plt.imshow(corr, interpolation='nearest', aspect='auto')
    plt.title("Matriz de correlación (numérica)")
    plt.xticks(range(len(num_cols)), num_cols, rotation=90)
    plt.yticks(range(len(num_cols)), num_cols)
    plt.colorbar()
    plt.tight_layout()
    plt.show()

    # Relación puntuaciones vs ventas
    plt.figure(figsize=(6,5))
    plt.scatter(df_clean["critic_score"], df_clean["global_sales"])
    plt.xlabel("Critic Score")
    plt.ylabel("Global Sales (M)")
    plt.title("Critic Score vs Ventas Globales")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6,5))
    plt.scatter(df_clean["user_score"], df_clean["global_sales"])
    plt.xlabel("User Score")
    plt.ylabel("Global Sales (M)")
    plt.title("User Score vs Ventas Globales")
    plt.tight_layout()
    plt.show()

In [ ]:
if 'df_clean' in globals() and set(["na_sales","eu_sales","jp_sales","other_sales"]).issubset(df_clean.columns):
    region_cols = ["na_sales","eu_sales","jp_sales","other_sales"]
    region_totals = df_clean[region_cols].sum().sort_values(ascending=False)
    display(region_totals)

    plt.figure(figsize=(7,5))
    region_totals.plot(kind='bar')
    plt.title("Ventas totales por región (M de copias)")
    plt.xlabel("Región")
    plt.ylabel("Ventas (millones)")
    plt.tight_layout()
    plt.show()

## 5) Guardado de dataset limpio